# ME4 Machine Learning - Tutorial 3

For simplicity, X-rays are often considered to have linear attenuation behaviour, which means that if the X-ray travels through a material a distance d, it will lose a particular fraction of its power, and if it travels through another distance d then it will lose the same fraction. This results in an exponential decay as the X-ray propagates. This is very convenient in many applications, particularly X-ray CT image generation, which relies on exactly this assumption. 

In practice, there is a phenomenon called ‘beam hardening’ which means that as an X-ray passes through an object, the beam becomes ‘harder’, i.e. more difficult to attenuate, so the attenuation value reduces. This means the behaviour will no longer be linear. We can try to address the challenges by finding a simple polynomial relationship, which is where regression becomes valuable.

In [ ]:
import sys
# setting path
sys.path.append('..')

from helperlib import (
    mathfunctions as hmath, 
    arrayfunctions as harray) 

import matplotlib.pyplot as plt
import numpy as np
import pandas
from sklearn import (
    datasets, 
    model_selection,
    naive_bayes)

np.random.seed(5)

# Don't show plots inline
%matplotlib agg

In [ ]:
fig = plt.figure(figsize=(6, 6))
grid = plt.GridSpec(2, 1)
ax1 = fig.add_subplot(grid[0, 0])
ax2 = fig.add_subplot(grid[1, 0], sharex=ax1)
fig.tight_layout(pad=5.0)

## 1. Standard linear regression

### 1.1. Import dataset

In [ ]:
df = pandas.read_csv('http://pogo.software/me4ml/xray.csv')

x = np.array(df['Distance (mm)'][:]) 
y = np.array(df['Total absorption'][:])